# Projeto de Turismo com LangChain - Passo 2: Construção das Cadeias Especializadas

Neste notebook, vamos criar cada uma das nossas cadeias (Chains) especializadas. Cada cadeia será um "especialista" em um tipo de tarefa:
1.  **Local Info Chain**: Fornece informações factuais sobre locais.
2.  **Logistics Chain**: Responde a perguntas sobre transporte e logística.
3.  **Itinerary Chain**: Cria roteiros de viagem personalizados.
4.  **Translation Chain**: Ajuda com frases úteis e traduções.

O objetivo é ter componentes modulares e testados antes de uni-los com um roteador no próximo passo.

## 0. Configuração Inicial

Primeiro, vamos importar as bibliotecas, carregar as variáveis de ambiente e inicializar os componentes que serão compartilhados por todas as cadeias:
- O **LLM (Groq)**, que será o cérebro de todas as cadeias.
- O **Modelo de Embeddings** e a conexão com o **Pinecone (VectorStore)**, que serão a base para nossas cadeias de RAG.

In [1]:
import os
from dotenv import load_dotenv

# LangChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings

# Carrega as variáveis do arquivo .env
load_dotenv()

# Pega as variáveis do ambiente
groq_api_key = os.getenv("GROQ_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name = os.getenv("PINECONE_INDEX_NAME")

# Validação das chaves
if not all([groq_api_key, pinecone_api_key, index_name]):
    print("Erro: Verifique se as variáveis GROQ_API_KEY, PINECONE_API_KEY e PINECONE_INDEX_NAME estão no arquivo .env")
else:
    print("Variáveis de ambiente carregadas com sucesso!")

/home/weriks/anaconda3/envs/sistema_turismo_langchain/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Variáveis de ambiente carregadas com sucesso!


/home/weriks/anaconda3/envs/sistema_turismo_langchain/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [5]:
# Inicialização dos componentes principais

# 1. LLM da Groq (Llama3)
llm = ChatGroq(
    model_name="llama-3.1-8b-instant",
    temperature=0.7, # Um pouco de criatividade, mas sem exageros
    api_key=groq_api_key
)
print("LLM (Groq) inicializado.")

# 2. Modelo de Embeddings (o mesmo usado na ingestão)
model_name = "sentence-transformers/all-MiniLM-L6-v2" 
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print("Modelo de Embeddings inicializado.")

# 3. Conexão com a base de dados vetorial (Pinecone)
# Usamos `from_existing_index` para nos conectarmos ao índice que já populamos
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
print(f"Conectado ao índice '{index_name}' no Pinecone.")

LLM (Groq) inicializado.
Modelo de Embeddings inicializado.
Conectado ao índice 'sistema-turismo' no Pinecone.


## Cadeia 1: Informações Locais (`LocalInfoChain`)

**Objetivo**: Responder a perguntas diretas sobre um ponto turístico, como horários, localização, etc.

**Mecanismo**: Esta cadeia usará RAG. Ela recebe um `contexto` (buscado do Pinecone) e a `pergunta` original do usuário. O prompt instrui o LLM a responder **com base estrita no contexto fornecido**.

In [6]:
# Criação do Prompt e da Cadeia de Informações Locais

local_info_prompt_template = """
Você é um assistente de turismo prestativo. Responda à pergunta do usuário de forma clara e concisa, utilizando APENAS as informações do contexto abaixo.
Se a resposta não estiver no contexto, diga "Desculpe, não encontrei essa informação na minha base de dados."

**Contexto:**
{context}

**Pergunta:**
{question}

**Resposta:**
"""

local_info_prompt = PromptTemplate(
    template=local_info_prompt_template,
    input_variables=["context", "question"]
)

local_info_chain = LLMChain(llm=llm, prompt=local_info_prompt, verbose=False)

print("LocalInfoChain criada com sucesso!")

LocalInfoChain criada com sucesso!


In [7]:
# Teste da LocalInfoChain

question_local = "Qual o horário de funcionamento do Cristo Redentor?"
print(f"Pergunta de teste: {question_local}")

# 1. Buscar contexto relevante no Pinecone
docs = vectorstore.similarity_search(question_local, k=3) # Busca os 3 chunks mais relevantes
context_local = "\n\n".join([doc.page_content for doc in docs])

# 2. Invocar a cadeia com o contexto e a pergunta
response = local_info_chain.invoke({
    "context": context_local,
    "question": question_local
})

print("\nResposta da Cadeia:")
print(response['text'])

Pergunta de teste: Qual o horário de funcionamento do Cristo Redentor?

Resposta da Cadeia:
Geralmente das 8h às 19h, mas pode variar. É essencial verificar o site oficial para horários e compra de ingressos antecipados para evitar filas.


## Cadeia 2: Logística de Transporte (`LogisticsChain`)

**Objetivo**: Responder perguntas de "como chegar", meios de transporte, etc.

**Mecanismo**: Similar à `LocalInfoChain`, utiliza RAG com um prompt focado em fornecer instruções claras de logística.

In [9]:
# Criação do Prompt e da Cadeia de Logística

logistics_prompt_template = """
Você é um especialista em logística de viagens. Responda à pergunta do usuário sobre transporte de forma direta e fácil de entender, usando as informações do contexto.

**Contexto:**
{context}

**Pergunta:**
{question}

**Instruções de Transporte:**
"""

logistics_prompt = PromptTemplate(
    template=logistics_prompt_template,
    input_variables=["context", "question"]
)

logistics_chain = LLMChain(llm=llm, prompt=logistics_prompt, verbose=False)

print("LogisticsChain criada com sucesso!")

LogisticsChain criada com sucesso!


In [11]:
# Teste da LogisticsChain

question_logistics = "Como eu chego na Torre Eiffel usando o metrô?"
print(f"Pergunta de teste: {question_logistics}")

# 1. Buscar contexto
docs = vectorstore.similarity_search(question_logistics, k=3)
context_logistics = "\n\n".join([doc.page_content for doc in docs])

# 2. Invocar a cadeia
response = logistics_chain.invoke({
    "context": context_logistics,
    "question": question_logistics
})

print("\nResposta da Cadeia:")
print(response['text'])

Pergunta de teste: Como eu chego na Torre Eiffel usando o metrô?

Resposta da Cadeia:
Para chegar à Torre Eiffel usando o metrô, você precisará seguir os seguintes passos:

1. **Metrô:** Pegue a linha 6 do metrô e desça na estação **Bir-Hakeim**. Essa estação fica bem próxima da Torre Eiffel e oferece uma vista impressionante do monumento.
2. **Saindo do Metrô:** Saindo da estação, você será direcionado a um ponto de vistas da Torre Eiffel. Se quiser uma vista melhor, pode seguir em direção à estação **Trocadéro**, que fica mais próxima da Torre Eiffel e oferece uma vista incrível do monumento e do rio Sena.
3. **Caminhada:** A partir dessa estação, você precisará caminhar alguns minutos até a Torre Eiffel. A caminhada é agradável e oferece uma visão da cidade.

Lembre-se de que é recomendável comprar ingressos online com antecedência para evitar filas enormes e aproveitar ao máximo sua visita à Torre Eiffel.


## Cadeia 3: Roteiros de Viagem (`ItineraryChain`)

**Objetivo**: Criar um roteiro de viagem com base nas preferências do usuário.

**Mecanismo**: Esta é a nossa cadeia mais criativa. Ela ainda usa RAG para obter informações sobre as atrações da cidade, mas o prompt é mais elaborado, pedindo ao LLM para agir como um agente de viagens e montar um plano detalhado.

In [12]:
# Criação do Prompt e da Cadeia de Roteiros

itinerary_prompt_template = """
Você é um agente de viagens experiente e criativo. Sua tarefa é criar um roteiro de viagem personalizado com base nas informações fornecidas.
Utilize o contexto para sugerir pontos turísticos e atividades relevantes.

**Contexto sobre o destino:**
{context}

**Detalhes da Viagem:**
- **Cidade:** {city}
- **Duração:** {days} dias
- **Perfil da Viagem:** {profile}

**Roteiro Detalhado (sugestão dia a dia):**
"""

itinerary_prompt = PromptTemplate(
    template=itinerary_prompt_template,
    input_variables=["context", "city", "days", "profile"]
)

itinerary_chain = LLMChain(llm=llm, prompt=itinerary_prompt, verbose=False)

print("ItineraryChain criada com sucesso!")

ItineraryChain criada com sucesso!


In [13]:
# Teste da ItineraryChain

# Detalhes da viagem para o teste
city_test = "Rio de Janeiro"
days_test = "2"
profile_test = "Focado em natureza e vistas panorâmicas"

print(f"Pedido de roteiro: {profile_test} para {city_test} por {days_test} dias.")

# 1. Buscar contexto sobre a cidade
# A busca é mais genérica para pegar várias atrações
docs = vectorstore.similarity_search(f"Pontos turísticos no {city_test}", k=5) 
context_itinerary = "\n\n".join([doc.page_content for doc in docs])

# 2. Invocar a cadeia
response = itinerary_chain.invoke({
    "context": context_itinerary,
    "city": city_test,
    "days": days_test,
    "profile": profile_test
})

print("\nResposta da Cadeia:")
print(response['text'])

Pedido de roteiro: Focado em natureza e vistas panorâmicas para Rio de Janeiro por 2 dias.

Resposta da Cadeia:
**Roteiro Detalhado para 2 Dias no Rio de Janeiro**

**Dia 1:**

1. **9h00 - 10h00:** Inicie o dia visitando a Praia de Copacabana, uma das praias mais famosas do mundo. Alugue uma cadeira de praia e aproveite o sol.
2. **10h30 - 12h30:** Dirija-se ao Morro do Corcovado, onde encontra a famosa estátua do Cristo Redentor. Pode ir de trem ou van oficial. Não recomendado ir de carro particular.
3. **13h00 - 14h00:** Desfrute do almoço em um restaurante localizado no bairro do Lapa, conhecido por sua arquitetura colonial e vida noturna.
4. **15h00 - 17h00:** Visite o Pão de Açúcar, um morro com vista panorâmica para a cidade e a Baía de Guanabara.
5. **19h00 - 20h00:** Jante em um restaurante de alta gastronomia na Zona Sul, conhecida por sua culinária internacional.

**Dia 2:**

1. **9h00 - 10h00:** Visite o Parque Nacional da Tijuca, um dos maiores parques urbanos do mundo, com

## Cadeia 4 (Bônus): Tradução (`TranslationChain`)

**Objetivo**: Fornecer frases úteis ou traduções simples para viajantes.

**Mecanismo**: Esta cadeia é diferente, pois **não usa RAG**. Ela depende apenas do conhecimento pré-treinado do LLM para realizar a tarefa. É uma `LLMChain` simples.

In [14]:
# Criação do Prompt e da Cadeia de Tradução

translation_prompt_template = """
Você é um guia de idiomas para viajantes. Responda à solicitação do usuário fornecendo frases úteis e suas traduções. Seja claro e direto.

**Solicitação do Usuário:**
{request}

**Frases e Traduções:**
"""

translation_prompt = PromptTemplate(
    template=translation_prompt_template,
    input_variables=["request"]
)

translation_chain = LLMChain(llm=llm, prompt=translation_prompt, verbose=False)

print("TranslationChain criada com sucesso!")

TranslationChain criada com sucesso!


In [15]:
# Teste da TranslationChain

request_translation = "Frases úteis para pedir um café e a conta em um restaurante em Paris."
print(f"Solicitação de teste: {request_translation}")

# Invocar a cadeia (sem necessidade de contexto)
response = translation_chain.invoke({"request": request_translation})

print("\nResposta da Cadeia:")
print(response['text'])

Solicitação de teste: Frases úteis para pedir um café e a conta em um restaurante em Paris.

Resposta da Cadeia:
Aqui estão algumas frases úteis para pedir um café e a conta em um restaurante em Paris:

1. **Pedir um café:**
 * "Pouvez-vous me donner un café, s'il vous plaît?" - Pode me dar um café, por favor?
 * "Je voudrais un café, s'il vous plaît." - Eu gostaria de um café, por favor.
 * "Un café, s'il vous plaît." - Um café, por favor.

2. **Pedir a conta:**
 * "L'addition, s'il vous plaît." - A conta, por favor.
 * "La note, s'il vous plaît." - A nota, por favor.
 * "Pouvez-vous me donner l'addition, s'il vous plaît?" - Pode me dar a conta, por favor?

3. **Agradecer:**
 * "Merci beaucoup." - Muito obrigado/a.
 * "Merci." - Obrigado/a.
 * "De rien." - De nada.

4. **Despedir-se:**
 * "Au revoir." - Até logo.
 * "À bientôt." - Até breve.
 * "Bonne journée." - Boa sorte.

Lembre-se de que em Paris, é comum dizer "s'il vous plaît" ao final das frases para mostrar cortesia. Além diss

## Conclusão

Excelente! Agora temos quatro cadeias especializadas, cada uma com seu próprio prompt e lógica, devidamente testadas. Estamos prontos para o próximo e último passo: construir o **RouterChain**, o componente que irá analisar a pergunta do usuário e direcioná-la para o especialista correto.